# Convergence Figure CRANE

### STEP 1. WOS Tranformation

- use citation-iot/ai-robot-clarivate.sql to extract WOS_id and WOS_ref
- add dataset column
- create a list of unique ids per target (wos_id) and source (ref_id) for a specific domain
- combine three datasets
- add 6 columns with binary values s.ai, s.iot, s.rob, t.ai, t.iot, t.rob. Use lists of ids and target ids to assign 1 or 0
- inspect and remove the self-looping source-target
    - Note: source-target-looping target_id does not have 'WOS:' at the beginning of the id. Remove those records.
- add year and citation information to wos_id using ISI files exported from MAC
- add year to ref_id
- add edge_id (combine wos and ref id separated by hyphen)

Create year-Source/Target dictionary (AItoROB) where the value is the number of Source/Target co-occurences 
Gephi nodes
- Rob Lat - 40
- Iot Lat - 20
- AI Lat - 60
- Longitude: (year - 1998) * 100

In [1]:
import pandas as pd
import numpy as np
import csv
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Load CSV Files
ai = pd.read_csv('../data/citation-ai-index-clarivate.csv', encoding='latin1', sep=",")
rob = pd.read_csv('../data/citation-robot-index-clarivate.csv', encoding='latin1', sep=",")
iot = pd.read_csv('../data/citation-iot-index-clarivate.csv', encoding='latin1', sep=",")

In [4]:
ai['dataset'] = 'ai'
rob['dataset'] = 'rob'
iot['dataset'] = 'iot'

In [5]:
ai_wos = set(ai.id)
ai_ref = set(ai.ref_id)
rob_wos = set(rob.id)
rob_ref = set(rob.ref_id)
iot_wos = set(iot.id)
iot_ref = set(iot.ref_id)

In [6]:
df = pd.concat([ai, rob, iot], ignore_index=True)

In [12]:
df.head()

,id,ref_id,dataset,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot
31,WOS:000403744000004,WOS:A1995TF15300001,ai,1,0,0,1,0,0
49,WOS:000397585800014,WOS:000378658500016,ai,1,0,0,1,0,0
51,WOS:000397585800014,WOS:000377173400001,ai,1,0,0,1,0,0
53,WOS:000397585800014,WOS:000379694100060,ai,1,0,0,1,0,0
54,WOS:000397585800014,WOS:000287620700007,ai,1,0,0,1,0,0


In [8]:
df['s.ai'] = df['id'].apply(lambda x: 1 if x in ai_wos else 0)
df['s.rob'] = df['id'].apply(lambda x: 1 if x in rob_wos else 0)
df['s.iot'] = df['id'].apply(lambda x: 1 if x in iot_wos else 0)

In [9]:
df['t.ai'] = df['ref_id'].apply(lambda x: 1 if x in ai_ref else 0)
df['t.rob'] = df['ref_id'].apply(lambda x: 1 if x in rob_ref else 0)
df['t.iot'] = df['ref_id'].apply(lambda x: 1 if x in iot_ref else 0)

In [11]:
df = df.drop(df[df['ref_id'].str.contains('^[0-9]',regex=True)==True].index)

### Upload ISI files

In [13]:
# Load CSV Files [I changed the column names '# Citations' > Citations, 'Publication Year' > Year, 'WoS ID' > wos_id]
ai_isi = pd.read_csv('../data/ISI-Publications-ai.csv', encoding='latin1', sep=",")
rob_isi = pd.read_csv('../data/ISI-Publications-Robotics.csv', encoding='latin1', sep=",")
iot_isi = pd.read_csv('../data/ISI-Publications-IOT.csv', encoding='latin1', sep=",")

In [14]:
df2 = pd.concat([ai_isi, rob_isi, iot_isi], ignore_index=True)

In [15]:
df3 = df.merge(df2, how='left', on=['id'])

In [20]:
df3.head()

,id,ref_id,dataset,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot,Year,Citations,edge_id
0,WOS:000403744000004,WOS:A1995TF15300001,ai,1,0,0,1,0,0,2017,11,WOS:000403744000004-WOS:A1995TF15300001
1,WOS:000397585800014,WOS:000378658500016,ai,1,0,0,1,0,0,2017,30,WOS:000397585800014-WOS:000378658500016
2,WOS:000397585800014,WOS:000377173400001,ai,1,0,0,1,0,0,2017,30,WOS:000397585800014-WOS:000377173400001
3,WOS:000397585800014,WOS:000379694100060,ai,1,0,0,1,0,0,2017,30,WOS:000397585800014-WOS:000379694100060
4,WOS:000397585800014,WOS:000287620700007,ai,1,0,0,1,0,0,2017,30,WOS:000397585800014-WOS:000287620700007


In [17]:
df3 = df3.drop(['Discipline','Subdiscipline', 'Title','Authors','Journal'], axis=1)

In [19]:
df3['edge_id'] = df3[['id', 'ref_id']].apply(lambda x: '-'.join(x.astype(str)), axis=1)

In [21]:
df4 = df3.drop(['ref_id','dataset', 's.ai','s.rob','s.iot','t.ai', 't.rob','t.iot','Citations','edge_id'], axis=1)

In [22]:
df4.tail()

,id,Year
506096,WOS:000401664100004,2016
506097,WOS:000401664100004,2016
506098,WOS:000401664100004,2016
506099,WOS:000401664100004,2016
506100,WOS:000401664100004,2016


In [23]:
df4.columns = ['ref_id', 'TargetYear']

In [24]:
df5 = pd.merge(df4,df3,  on=['ref_id'])

In [32]:
df5 = pd.merge(df4,df3,  on=['ref_id'])
df5 = df5.drop_duplicates()

In [33]:
df5.head()

,ref_id,TargetYear,id,dataset,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot,Year,Citations,edge_id
0,WOS:000289029800005,2011,WOS:000385595200022,ai,1,0,0,1,0,0,2016,15,WOS:000385595200022-WOS:000289029800005
36,WOS:000291123500010,2011,WOS:000403632000005,ai,1,0,0,1,0,0,2017,2,WOS:000403632000005-WOS:000291123500010
113,WOS:000413946600014,2017,WOS:000416790500071,iot,0,0,1,0,0,1,2017,4,WOS:000416790500071-WOS:000413946600014
150,WOS:000290536400012,2011,WOS:000372644300043,ai,1,0,0,1,0,0,2016,1,WOS:000372644300043-WOS:000290536400012
161,WOS:000400822900194,2017,WOS:000408751100012,ai,1,0,0,1,0,0,2017,3,WOS:000408751100012-WOS:000400822900194


In [34]:
df5.columns.tolist()

['ref_id',
 'TargetYear',
 'id',
 'dataset',
 's.ai',
 's.rob',
 's.iot',
 't.ai',
 't.rob',
 't.iot',
 'Year',
 'Citations',
 'edge_id']

In [35]:
cols = ['edge_id', 'id','ref_id', 's.ai','s.rob','s.iot','t.ai','t.rob','t.iot','dataset','Year','TargetYear','Citations']

In [36]:
df5 = df5[cols]

In [37]:
df5.to_csv('convergence_id_ref.csv', index=False)

### Step 2. Conver to Network
- import convergence_id_ref.csv
- parse df3
- create Source and Target nodes and edges

In [2]:
df5 = pd.read_csv('convergence_id_ref.csv', encoding='latin1', sep=",")

In [3]:
df5 = df5.drop_duplicates()

In [4]:
len(df5.id)

29565

In [10]:
df5.to_csv('convergence.csv', index=False)

In [5]:
df5.head()

,edge_id,id,ref_id,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot,dataset,Year,TargetYear,Citations
0,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
1,WOS:000403632000005-WOS:000291123500010,WOS:000403632000005,WOS:000291123500010,1,0,0,1,0,0,ai,2017,2011,2
2,WOS:000416790500071-WOS:000413946600014,WOS:000416790500071,WOS:000413946600014,0,0,1,0,0,1,iot,2017,2017,4
3,WOS:000372644300043-WOS:000290536400012,WOS:000372644300043,WOS:000290536400012,1,0,0,1,0,0,ai,2016,2011,1
4,WOS:000408751100012-WOS:000400822900194,WOS:000408751100012,WOS:000400822900194,1,0,0,1,0,0,ai,2017,2017,3


In [6]:
wosYears = {}
wosCitations = {}
AItoROB = {}
IOTtoAI = {}
ROBtoIOT = {}
ROBtoAI = {}
AItoIOT = {}
IOTtoROB = {}

Nodes = []
NodeDict = {}
Edges = []

In [5]:
df5.head()

,edge_id,id,ref_id,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot,dataset,Year,TargetYear,Citations
0,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
1,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
2,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
3,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
4,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15


In [7]:
for index, row in df5.iterrows():
    wosID = str(row[1])
    year = str(row[10])
    wosYears[wosID] = year

In [12]:
#wosYears = dict(zip(df5.id, df5.Year))

In [45]:
len(wosYears)

11018

In [75]:
#AItoROB

In [8]:
for index, row in df5.iterrows():
    AIsource = row['s.ai']
    AItarget = row['t.ai']
    ROBsource = row['s.rob']
    ROBtarget = row['t.rob']
    IOTsource = row['s.iot']
    IOTtarget = row['t.iot']
    srcYear = row['Year']#wosYears[row[1]]
    trgYear = row['TargetYear']
    if ((AIsource == 1) & (ROBtarget == 1)):
        #AItoROB[(srcYear, trgYear)] = 1
        keys = (srcYear, trgYear)
        AItoROB[keys] = AItoROB.setdefault(keys, 0) + 1
       # print(keys)
        #if AItoROB.keys in ((srcYear, trgYear)):
           # print(srcYear)
         #   AItoROB[(srcYear, trgYear)] += 1
            
       # else: AItoROB[(srcYear, trgYear)] = 1

    if ((ROBsource == 1) & (IOTtarget == 1)):
        #print((srcYear, trgYear))
        keys = (srcYear, trgYear)
        ROBtoIOT[keys] = ROBtoIOT.setdefault(keys, 0) + 1
       # if ROBtoIOT.keys in ((srcYear, trgYear)):
        #    ROBtoIOT[(srcYear, trgYear)] += 1
        #else: ROBtoIOT[(srcYear, trgYear)] = 1
    if ((IOTsource == 1) & (AItarget == 1)):
        #if IOTtoAI.keys in ((srcYear, trgYear)):
        ##    IOTtoAI[(srcYear, trgYear)] += 1
       # else: IOTtoAI[(srcYear, trgYear)] = 1
        keys = (srcYear, trgYear)
        IOTtoAI[keys] = IOTtoAI.setdefault(keys, 0) + 1
    if ((ROBsource == 1) & (AItarget == 1)):
       # if ROBtoAI.keys in ((srcYear, trgYear)):
       #     ROBtoAI[(srcYear, trgYear)] += 1
       # else: ROBtoAI[(srcYear, trgYear)] = 1
        keys = (srcYear, trgYear)
        ROBtoAI[keys] = ROBtoAI.setdefault(keys, 0) + 1
    if ((AIsource == 1) & (IOTtarget == 1)):
       # if AItoIOT.keys in ((srcYear, trgYear)):
        #    AItoIOT[(srcYear, trgYear)] += 1
       # else: AItoIOT[(srcYear, trgYear)] = 1
        keys = (srcYear, trgYear)
        AItoIOT[keys] = AItoIOT.setdefault(keys, 0) + 1
    if ((IOTsource == 1) & (ROBtarget == 1)):
      #  if IOTtoROB.keys in ((srcYear, trgYear)):
       #     IOTtoROB[(srcYear, trgYear)] += 1
      #  else: IOTtoROB[(srcYear, trgYear)] = 1
        keys = (srcYear, trgYear)
        IOTtoROB[keys] = IOTtoROB.setdefault(keys, 0) + 1
   # print(row['c1'], row['c2'])


In [46]:
nodes = []
for key, value in AItoROB.items():
    nodes.append([key[0],key[1],value])
   # print(key[0],key[1],value)

In [47]:
pd.DataFrame(nodes).to_csv("ai_to_rob_nodes.csv",index=False)

In [56]:
for key, value in ROBtoIOT.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
    print(int(key[0]), int(key[1]), (int(key[0]) - 1998), ((int(key[0]) - 1998)*100), ((int(key[1]) - 1998)*100))
print(Nodes)
print(Edges)

2016 2016 18 1800 1800
2017 2012 19 1900 1400
2016 2009 18 1800 1100
2010 2007 12 1200 900
2012 2005 14 1400 700
2011 2005 13 1300 700
2010 2005 12 1200 700
2015 2005 17 1700 700
2013 2005 15 1500 700
2008 2005 10 1000 700
2017 2005 19 1900 700
2009 2005 11 1100 700
2014 1999 16 1600 100
2014 2012 16 1600 1400
2016 2013 18 1800 1500
2015 2013 17 1700 1500
2014 2013 16 1600 1500
2017 2013 19 1900 1500
2017 2015 19 1900 1700
2016 2015 18 1800 1700
2015 2015 17 1700 1700
2016 2014 18 1800 1600
2007 2004 9 900 600
2017 2016 19 1900 1800
2004 2004 6 600 600
2008 2004 10 1000 600
2011 2004 13 1300 600
2006 2004 8 800 600
2012 2004 14 1400 600
2010 2004 12 1200 600
2009 2004 11 1100 600
2016 2004 18 1800 600
2014 2004 16 1600 600
2013 2004 15 1500 600
2015 2004 17 1700 600
2011 2007 13 1300 900
2007 2007 9 900 900
2017 2007 19 1900 900
2016 2007 18 1800 900
2012 2007 14 1400 900
2015 2007 17 1700 900
2013 2007 15 1500 900
2014 2007 16 1600 900
2009 2006 11 1100 800
2011 2006 13 1300 800
2015 

In [77]:
for key, value in ROBtoIOT.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in IOTtoROB.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 20, srcLon]) #IoT row
    Nodes.append([len(Nodes) + 1, int(key[1]), 40, trgLon]) #ROB row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in AItoIOT.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 60, srcLon]) #AI row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in IOTtoAI.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 20, srcLon]) #IoT row
    Nodes.append([len(Nodes) + 1, int(key[1]), 60, trgLon]) #AI row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in AItoROB.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 60, srcLon]) #AI row
    Nodes.append([len(Nodes) + 1, int(key[1]), 40, trgLon]) #ROB row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in ROBtoAI.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 60, trgLon]) #AI row
    Edges.append([len(Nodes) - 1, len(Nodes), value])

for node in Nodes:
    NodeDict[node[0]] = node[2] # create dictionary of ID-Latitude pairs

with open('WOS_convergence_edges.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["source", "target", "weight", "cat"])
    for edge in Edges:
        if (NodeDict[edge[0]] == 20): # IoT
            edge.append(0)
            writer.writerow(edge)
        elif (NodeDict[edge[0]] == 40): # Robotics
            edge.append(1)
            writer.writerow(edge)
        else:                          # AI
            edge.append(2)
            writer.writerow(edge)

with open('WOS_convergence_nodes.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "label", "lat", "lon"])
    for node in Nodes:
        writer.writerow(node)

## Try without diff in years

In [48]:
Nodes = []
NodeDict = {}
Edges = []

In [49]:
for key, value in ROBtoIOT.items():
    srcLon = int(key[0])
    trgLon = int(key[1])
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in IOTtoROB.items():
    srcLon = int(key[0])
    trgLon = int(key[1])
    Nodes.append([len(Nodes) + 1, int(key[0]), 20, srcLon]) #IoT row
    Nodes.append([len(Nodes) + 1, int(key[1]), 40, trgLon]) #ROB row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in AItoIOT.items():
    srcLon = int(key[0])
    trgLon = int(key[1])
    Nodes.append([len(Nodes) + 1, int(key[0]), 60, srcLon]) #AI row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in IOTtoAI.items():
    srcLon = int(key[0])
    trgLon = int(key[1])
    Nodes.append([len(Nodes) + 1, int(key[0]), 20, srcLon]) #IoT row
    Nodes.append([len(Nodes) + 1, int(key[1]), 60, trgLon]) #AI row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in AItoROB.items():
    srcLon = int(key[0])
    trgLon = int(key[1])
    Nodes.append([len(Nodes) + 1, int(key[0]), 60, srcLon]) #AI row
    Nodes.append([len(Nodes) + 1, int(key[1]), 40, trgLon]) #ROB row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in ROBtoAI.items():
    srcLon = int(key[0])
    trgLon = int(key[1])
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 60, trgLon]) #AI row
    Edges.append([len(Nodes) - 1, len(Nodes), value])




In [64]:
print(Nodes[1:10])
print(Edges[1:10])

[[2, 2016, 20, 2016], [3, 2017, 40, 2017], [4, 2012, 20, 2012], [5, 2016, 40, 2016], [6, 2009, 20, 2009], [7, 2010, 40, 2010], [8, 2007, 20, 2007], [9, 2012, 40, 2012], [10, 2005, 20, 2005]]
[[3, 4, 10], [5, 6, 3], [7, 8, 5], [9, 10, 7], [11, 12, 4], [13, 14, 4], [15, 16, 8], [17, 18, 2], [19, 20, 4]]


In [50]:

for node in Nodes:
    NodeDict[node[0]] = node[2] # create dictionary of ID-Latitude pairs

In [51]:
with open('WOS_convergence_edges.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["source", "target", "weight", "cat"])
    for edge in Edges:
        if (NodeDict[edge[0]] == 20): # IoT
            edge.append(0)
            writer.writerow(edge)
        elif (NodeDict[edge[0]] == 40): # Robotics
            edge.append(1)
            writer.writerow(edge)
        else:                          # AI
            edge.append(2)
            writer.writerow(edge)

with open('WOS_convergence_nodes.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "label", "lat", "lon"])
    for node in Nodes:
        writer.writerow(node)

In [78]:
#print("AI->ROB: " + str(AItoROB))
#print("ROB->IOT: " + str(ROBtoIOT))
#print("IOT->AI: " + str(IOTtoAI))
#print("ROB->AI: " + str(ROBtoAI))
#print("AI->IOT: " + str(AItoIOT))
#print("IOT->ROB : " + str(IOTtoROB))

In [15]:
srcYear = dict(zip(df5.id, df5.Year))
trgYear = dict(zip(df5.id, df5.TargetYear))

In [34]:
if ((AIsource == '1') & (ROBtarget == '1')):
    if AItoROB.has_key((srcYear, trgYear)):
        AItoROB[(srcYear, trgYear)] += 1
    else: AItoROB[(srcYear, trgYear)] = 1
if ((ROBsource == '1') & (IOTtarget == '1')):
    if ROBtoIOT.has_key((srcYear, trgYear)):
        ROBtoIOT[(srcYear, trgYear)] += 1
    else: ROBtoIOT[(srcYear, trgYear)] = 1
if ((IOTsource == '1') & (AItarget == '1')):
    if IOTtoAI.has_key((srcYear, trgYear)):
        IOTtoAI[(srcYear, trgYear)] += 1
    else: IOTtoAI[(srcYear, trgYear)] = 1
if ((ROBsource == '1') & (AItarget == '1')):
    if ROBtoAI.has_key((srcYear, trgYear)):
        ROBtoAI[(srcYear, trgYear)] += 1
    else: ROBtoAI[(srcYear, trgYear)] = 1
if ((AIsource == '1') & (IOTtarget == '1')):
    if AItoIOT.has_key((srcYear, trgYear)):
        AItoIOT[(srcYear, trgYear)] += 1
    else: AItoIOT[(srcYear, trgYear)] = 1
if ((IOTsource == '1') & (ROBtarget == '1')):
    if IOTtoROB.has_key((srcYear, trgYear)):
        IOTtoROB[(srcYear, trgYear)] += 1
    else: IOTtoROB[(srcYear, trgYear)] = 1

In [40]:
for row in df5:
if ((AIsource == '1') & (ROBtarget == '1')):
    if AItoROB.has_key((srcYear, trgYear)):
        AItoROB[(srcYear, trgYear)] += 1
    else: AItoROB[(srcYear, trgYear)] = 1

{}

In [39]:
for key, value in ROBtoIOT.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in IOTtoROB.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 20, srcLon]) #IoT row
    Nodes.append([len(Nodes) + 1, int(key[1]), 40, trgLon]) #ROB row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in AItoIOT.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 60, srcLon]) #AI row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in IOTtoAI.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 20, srcLon]) #IoT row
    Nodes.append([len(Nodes) + 1, int(key[1]), 60, trgLon]) #AI row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in AItoROB.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 60, srcLon]) #AI row
    Nodes.append([len(Nodes) + 1, int(key[1]), 40, trgLon]) #ROB row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in ROBtoAI.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 60, trgLon]) #AI row
    Edges.append([len(Nodes) - 1, len(Nodes), value])

for node in Nodes:
    NodeDict[node[0]] = node[2] # create dictionary of ID-Latitude pairs

with open('WOS_convergence_edges.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["source", "target", "weight", "cat"])
    for edge in Edges:
        if (NodeDict[edge[0]] == 20): # IoT
            edge.append(0)
            writer.writerow(edge)
        elif (NodeDict[edge[0]] == 40): # Robotics
            edge.append(1)
            writer.writerow(edge)
        else:                          # AI
            edge.append(2)
            writer.writerow(edge)

with open('WOS_convergence_nodes.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "label", "lat", "lon"])
    for node in Nodes:
        writer.writerow(node)